In [1]:
#Impoty of libraries
import pandas as pd
import numpy as np

<h2>Week 3</h2>

<h2>Task 1</h2>

In [3]:
html_page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(html_page)

In [4]:
df = dfs[0]
df.describe()
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


1. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
#Exclude boroughs with values "Not assigned"
df = df[df['Borough']!='Not assigned']
df_backup = df

In [6]:
# Resetting the index
df.index = range(len(df.index))
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [7]:
idx_to_change = df.loc[df['Neighbourhood'] == "Not assigned"].index

# Iterate the list of indicies and set Column2 to the value of Column1 at the given index
for i in idx_to_change:
    df.iloc[i, 2] = df.iloc[i, 1] 
df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.


In [8]:
# Merging rows with same Postalcode

df1= df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list)
df2= df1.sample(frac=1).reset_index()
df2['Neighbourhood'] = df2['Neighbourhood'].str.join(', ')
df2.head()

,Postcode,Borough,Neighbourhood
0,M5P,Central Toronto,"Forest Hill North, Forest Hill West"
1,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto"
2,M3B,North York,Don Mills North
3,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol..."
4,M9A,Queen's Park,Queen's Park


In [9]:
print(df2.shape)

(103, 3)


<h3>Step 2: Loading of coordinates data</h3>
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [10]:
# Load coordinate data from CSV
url_geo = 'http://cocl.us/Geospatial_data'
df_geo = pd.read_csv(url_geo)
#Renaming of column
df_geo = df_geo.rename({'Postal Code':'Postcode'}, axis=1)
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df = df2.merge(df_geo)
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
1,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321
2,M3B,North York,Don Mills North,43.745906,-79.352188
3,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201
4,M9A,Queen's Park,Queen's Park,43.667856,-79.532242


In [12]:
df.tail(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
93,M6G,Downtown Toronto,Christie,43.669542,-79.422564
94,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
95,M4E,East Toronto,The Beaches,43.676357,-79.293031
96,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999
97,M6B,North York,Glencairn,43.709577,-79.445073
98,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
99,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
100,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
101,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
102,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476


<h3>Step 3</h3>
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1.  to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan